## An Example of Data Cleaning with Pandera

In [1]:
import pandas as pd
import pandera as pa


## Read Raw Data

In [2]:
fp = "../../data/online_retail_II.csv"
df = pd.read_csv(fp)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_7354/3119532506.py:2: DtypeWarning: Columns (3,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)


### Note:
1. You already see warnings about mixed data types
2. Inspect the data set to create expected range of values for each attribute - create sensible obvious rules

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/09 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/09 7:45,2.1,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/09 7:45,1.25,13085.0,United Kingdom


In [4]:
df.shape

(1067372, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067372 entries, 0 to 1067371
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   Invoice      1067372 non-null  object
 1   StockCode    1067372 non-null  object
 2   Description  1062990 non-null  object
 3   Quantity     1067372 non-null  object
 4   InvoiceDate  1067372 non-null  object
 5   Price        1067372 non-null  object
 6   Customer ID  824365 non-null   object
 7   Country      1067372 non-null  object
dtypes: object(8)
memory usage: 65.1+ MB


### Note
1. Inspection of the info() call tells us that Customer ID has null values
2. Generate an inferred schema to see what the inferred data types are for each attribute. The object category result for fields where we expect numeric values indicates data errors. We will have to use pandera to catch these errors

In [6]:
pa.infer_schema(df)

<Schema DataFrameSchema(columns={'Invoice': <Schema Column(name=Invoice, type=DataType(object))>, 'StockCode': <Schema Column(name=StockCode, type=DataType(object))>, 'Description': <Schema Column(name=Description, type=DataType(object))>, 'Quantity': <Schema Column(name=Quantity, type=DataType(object))>, 'InvoiceDate': <Schema Column(name=InvoiceDate, type=DataType(object))>, 'Price': <Schema Column(name=Price, type=DataType(object))>, 'Customer ID': <Schema Column(name=Customer ID, type=DataType(object))>, 'Country': <Schema Column(name=Country, type=DataType(object))>}, checks=[], index=<Schema Index(name=None, type=DataType(int64))>, coerce=True, dtype=None, strict=False, name=None, ordered=False, unique_column_names=Falsemetadata='None, unique_column_names=False, add_missing_columns=False)>

In [7]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

## Validation against expected range of values

For each attribute in the columns:


    1. For each attribute, define the range of values you expect to see with a schema
    
    2. Apply the schema (validate it)
    
    3. Analyze the results
    


In [8]:
from pandera.errors import SchemaErrors
invoice_schema = pa.SeriesSchema(
    str,
    coerce=True,
    nullable=False,
    name="Invoice")
try:
    validated_invoices = invoice_schema.validate(df["Invoice"])
    print("Invoice data has values that are as expected")
except SchemaErrors as se:
    print("Invoice data needs analysis - has values that are unexpected!")
    print("Failure Cases:")
    print(se.failure_cases)

Invoice data has values that are as expected


In [9]:
stock_code_schema = pa.SeriesSchema(
    str,
    [pa.Check.str_matches(r'^[0-9A-Za-z]{4,}$')],
    coerce=True,
    nullable=False,
    name="StockCode")

In [10]:
failure_cases = None
try:
    validated_stock_code = stock_code_schema.validate(df["StockCode"])
    print("StockCode data has values that are as expected")
except pa.errors.SchemaErrors as err:
    print("StockCode data needs analysis - has values that are unexpected!")
    failure_cases = err.failure_cases
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)
    

SchemaError:          index failure_case
0         735            D
1         736            D
2        2379          DOT
3        2539          DOT
4        2551          DOT
...       ...          ...
3640  1063783            M
3641  1064830          DOT
3642  1066370          DOT
3643  1067002          DOT
3644  1067003            M

[3645 rows x 2 columns]


In [11]:
description_code_schema = pa.SeriesSchema(
    str,
    coerce=True,
    nullable=False,
    name="Description")

In [12]:
failure_cases = None
try:
    description_code_validation = description_code_schema.validate(df["Description"])
    print("Description data has values that are as expected")
# except pa.errors.SchemaErrors as err:
#     print("Description data needs analysis - has values that are unexpected!")
#     failure_cases = err.failure_cases
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

SchemaError:          index failure_case
0         470          NaN
1        3114          NaN
2        3161          NaN
3        3731          NaN
4        4296          NaN
...       ...          ...
4377  1060784          NaN
4378  1060788          NaN
4379  1060794          NaN
4380  1062443          NaN
4381  1064016          NaN

[4382 rows x 2 columns]


In [13]:
quantity_schema = pa.SeriesSchema(
    int,
    [pa.Check.greater_than(0)],
    coerce=True,
    nullable=False,
    name="Quantity")

In [14]:
failure_cases = None
try:
    quantity_validation = quantity_schema.validate(df["Quantity"])
    print("Quantity data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

SchemaError:      index failure_case
0  525461     Quantity


### Note
This is a particularly "nasty" error. In a dataset with over 1 M rows, there is one row that has values where Quantity has the value "Quantity" instead of a number and other attributes like "InvoiceDate" also have bad values. Removing this row eliminates this issue. 

In [15]:
df["Quantity"][525461]

'Quantity'

In [16]:
invoice_date_schema = pa.SeriesSchema(
    pd.DatetimeTZDtype(unit="ns", tz="UTC"),
    coerce=True,
    nullable=False,
    name="InvoiceDate")

In [ ]:
failure_cases = None
try:
    invoice_date_validation = invoice_date_schema.validate(df["InvoiceDate"])
    print("InvoiceDate data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

/Users/rajivsambasivan/opt/anaconda3/envs/init_r2ds/lib/python3.10/site-packages/pandera/engines/pandas_engine.py:842: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  col = to_datetime_fn(col, **self.to_datetime_kwargs)


In [ ]:
df["InvoiceDate"][525461]

In [ ]:
df = df.drop(index=525461)


In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
failure_cases = None
try:
    quantity_validation = quantity_schema.validate(df["Quantity"])
    print("Quantity data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

In [ ]:
failure_cases = None
try:
    invoice_date_validation = invoice_date_schema.validate(df["InvoiceDate"])
    print("InvoiceDate data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

In [ ]:
price_schema = pa.SeriesSchema(
    float,
    [pa.Check.greater_than(0)],
    coerce=True,
    nullable=False,
    name="Price")

In [ ]:
failure_cases = None
try:
    price_validation = price_schema.validate(df["Price"])
    print("Price data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

In [ ]:
customer_schema = pa.SeriesSchema(
    str,
    coerce=True,
    nullable=False,
    name="Customer ID")

In [ ]:
failure_cases = None
try:
    customer_validation = customer_schema.validate(df["Customer ID"])
    print("Customer ID data has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

In [ ]:
country_schema = pa.SeriesSchema(
    str,
    coerce=True,
    nullable=False,
    name="Country")

In [ ]:
failure_cases = None
try:
    country_validation = country_schema.validate(df["Country"])
    print("Countrydata has values that are as expected")
except pa.errors.SchemaError as err:
    print('SchemaError: ', err.failure_cases)

## Conclusion
The results we obtained through initial assessment of data quality inform the "denoising" step. This notebook illustrates a systematic way of "denoising" data with data quality tools such as [pandera](https://union.ai/pandera)